# Lesson 4 - Quantization


In this lesson, we'll discuss the concept of "quantization". This technique helps reduce the memory overhead of a model and enables running inference with larger LLMs.

这个lesson主要会学习量化的工作原理，并且实现一个算法将LM量化为8bit，并且在推理期间解量化

### Import required packages and load the LLM

In [2]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from utils import generate

In [3]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# pad on the left so we can append new tokens on the right
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

### Define a Float 32 type

In [5]:
# fix dtype post quantization to "pretend" to be fp32
def get_float32_dtype(self):
    return torch.float32
GPT2Model.dtype = property(get_float32_dtype)

In [6]:
model.get_memory_footprint()

510342192

### Define a quantization function

In [7]:
# 这个函数的作用是将一个浮点数张量进行量化处理，将其值映射到0-255之间的无符号8位整数
def quantize(t):
    '''
    Quantizes a 2D tensor to uint8.
    Parameters
    ----------
    t : torch.Tensor
        The input tensor to be quantized. It should be a 2D tensor with shape (batch_size, sequence_length).
    Returns
    ------- 
    '''
    # obtain range of values in the tensor to map between 0 and 255
    min_val, max_val = t.min(), t.max()

    # determine the "zero-point", or value in the tensor to map to 0
    scale = (max_val - min_val) / 255
    zero_point = min_val

    # quantize and clamp to ensure we're in [0, 255]
    t_quant = (t - zero_point) / scale
    t_quant = torch.clamp(t_quant, min=0, max=255)

    # keep track of scale and zero_point for reversing quantization
    state = (scale, zero_point)

    # cast to uint8 and return
    t_quant = t_quant.type(torch.uint8)
    return t_quant, state

In [8]:
t = model.transformer.h[0].attn.c_attn.weight.data
print(t, t.shape)

tensor([[-0.4738, -0.2614, -0.0978,  ...,  0.0513, -0.0584,  0.0250],
        [ 0.0874,  0.1473,  0.2387,  ..., -0.0525, -0.0113, -0.0156],
        [ 0.0039,  0.0695,  0.3668,  ...,  0.1143,  0.0363, -0.0318],
        ...,
        [-0.2592, -0.0164,  0.1991,  ...,  0.0095, -0.0516,  0.0319],
        [ 0.1517,  0.2170,  0.1043,  ...,  0.0293, -0.0429, -0.0475],
        [-0.4100, -0.1924, -0.2400,  ..., -0.0046,  0.0070,  0.0198]]) torch.Size([768, 2304])


In [9]:
t_q, state = quantize(t)
print(t_q, t_q.min(), t_q.max())

tensor([[107, 116, 124,  ..., 130, 125, 129],
        [132, 135, 139,  ..., 126, 128, 127],
        [128, 131, 145,  ..., 133, 130, 127],
        ...,
        [116, 127, 137,  ..., 129, 126, 130],
        [135, 138, 133,  ..., 129, 126, 126],
        [110, 119, 117,  ..., 128, 128, 129]], dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8)


### Define a dequantization function

In [10]:
# 这个函数的作用是将一个量化后的无符号8位整数张量进行反量化处理，将其值映射回原始的浮点数范围
def dequantize(t, state):
    scale, zero_point = state
    return t.to(torch.float32) * scale + zero_point

In [11]:
t_rev = dequantize(t_q, state)
print(t_rev)

tensor([[-0.4774, -0.2783, -0.1014,  ...,  0.0313, -0.0793,  0.0092],
        [ 0.0755,  0.1419,  0.2303,  ..., -0.0572, -0.0129, -0.0351],
        [-0.0129,  0.0534,  0.3630,  ...,  0.0976,  0.0313, -0.0351],
        ...,
        [-0.2783, -0.0351,  0.1861,  ...,  0.0092, -0.0572,  0.0313],
        [ 0.1419,  0.2082,  0.0976,  ...,  0.0092, -0.0572, -0.0572],
        [-0.4110, -0.2120, -0.2562,  ..., -0.0129, -0.0129,  0.0092]])


In [ ]:
# 计算量化和反量化后的张量之间的差异
torch.abs(t - t_rev)

tensor([[0.0035, 0.0170, 0.0036,  ..., 0.0200, 0.0209, 0.0158],
        [0.0119, 0.0055, 0.0084,  ..., 0.0046, 0.0017, 0.0195],
        [0.0168, 0.0161, 0.0038,  ..., 0.0167, 0.0050, 0.0032],
        ...,
        [0.0191, 0.0187, 0.0131,  ..., 0.0004, 0.0056, 0.0006],
        [0.0098, 0.0088, 0.0067,  ..., 0.0202, 0.0143, 0.0097],
        [0.0010, 0.0196, 0.0162,  ..., 0.0084, 0.0199, 0.0107]])

In [13]:
response_expected = generate(
    model,
    tokenizer,
    [("The quick brown fox jumped over the", 10)]
)[0]
response_expected

'The quick brown fox jumped over the fence and ran to the other side of the fence'

### Let's apply the quantization technique to the entire model

In [15]:
def quantize_model(model):
    states = {}
    for name, param in model.named_parameters():
        param.requires_grad = False
        param.data, state = quantize(param.data)
        states[name] = state
    return model, states
''' 
The `quantize_model` function is designed to apply quantization to all the parameters of a given model. Quantization is a technique used to reduce the memory footprint and computational requirements of a model by converting its parameters (typically stored as high-precision floating-point numbers) into lower-precision representations, such as 8-bit integers. Here's a detailed explanation of the function:

1. **Initialization**:  
   The function initializes an empty dictionary called `states`. This dictionary will store the quantization metadata (scale and zero-point) for each parameter in the model. These values are necessary for reversing the quantization process (dequantization) if needed later.

2. **Iterating Over Model Parameters**:  
   The function loops through all the parameters of the model using `model.named_parameters()`. This method returns both the name and the parameter tensor for each parameter in the model. The loop ensures that every parameter is processed for quantization.

3. **Freezing Gradients**:  
   For each parameter, `param.requires_grad` is set to `False`. This step ensures that the parameter is not updated during training, as quantized models are typically used for inference rather than training. Freezing gradients also prevents unnecessary computation during backpropagation.

4. **Quantizing Parameters**:  
   The `quantize` function is called on the parameter's data (`param.data`). This function converts the parameter tensor into an 8-bit integer representation and returns the quantized tensor along with its quantization metadata (scale and zero-point). The quantized tensor replaces the original parameter data, and the metadata is stored in the `states` dictionary under the parameter's name.

5. **Returning the Quantized Model**:  
   After all parameters have been quantized, the function returns the modified model (with quantized parameters) and the `states` dictionary. The `states` dictionary is crucial for dequantization or debugging, as it contains the information needed to reconstruct the original floating-point values.

### Key Concepts:
- **Quantization**: The `quantize` function reduces the precision of the model's parameters by mapping their values to a range of integers (0–255). This reduces memory usage and speeds up inference, especially on hardware optimized for integer operations.
- **Scale and Zero-Point**: These values are used to map the original floating-point range of the tensor to the integer range. They are stored in `states` to allow for accurate dequantization later.
- **Inference Optimization**: Quantized models are typically used in scenarios where inference speed and resource efficiency are critical, such as deploying models on edge devices or mobile platforms.

This function is a key step in preparing a model for efficient deployment, ensuring that it uses fewer resources while maintaining acceptable accuracy.
'''


" \nThe `quantize_model` function is designed to apply quantization to all the parameters of a given model. Quantization is a technique used to reduce the memory footprint and computational requirements of a model by converting its parameters (typically stored as high-precision floating-point numbers) into lower-precision representations, such as 8-bit integers. Here's a detailed explanation of the function:\n\n1. **Initialization**:  \n   The function initializes an empty dictionary called `states`. This dictionary will store the quantization metadata (scale and zero-point) for each parameter in the model. These values are necessary for reversing the quantization process (dequantization) if needed later.\n\n2. **Iterating Over Model Parameters**:  \n   The function loops through all the parameters of the model using `model.named_parameters()`. This method returns both the name and the parameter tensor for each parameter in the model. The loop ensures that every parameter is processed 

In [16]:
quant_model, states = quantize_model(model)

In [17]:
quant_model.get_memory_footprint()

137022768

In [19]:
def size_in_bytes(t):
    return t.numel() * t.element_size()



In [20]:
# 计算记录metadata的内存占用
sum([
    size_in_bytes(v[0]) + size_in_bytes(v[1])
    for v in states.values()
])

1184

In [21]:
def dequantize_model(model, states):
    for name, param in model.named_parameters():
        state = states[name]
        param.data = dequantize(param.data, state)
    return model

In [22]:
dequant_model = dequantize_model(quant_model, states)

In [23]:
dequant_model.get_memory_footprint()

510342192

In [25]:
response_expected = generate(
    dequant_model, # use dequantized model
    tokenizer,
    [("The quick brown fox jumped over the", 10)]
)[0]
response_expected

'The quick brown fox jumped over the fence.\n\nThe fox jumped over the fence'